# Post processing scraped data

This notebook processes the scraped data from Portal de la Reserca to create the Nodelist and Edgelist to plot in Gephi.

# Import modules

In [11]:
import numpy as np
import pandas as pd
from ast import literal_eval

# Get researchers of interest

In [12]:
res_df = pd.read_csv('./data/nodelist.csv')
res_df_IGTP = res_df.loc[res_df['institution'] == 'IGTP']
res_IGTP = res_df_IGTP['id'].unique()

# Get edgelist

In [13]:
# papers0_df = pd.read_csv('./data/papers_0.csv')
# papers1_df = pd.read_csv('./data/papers_1.csv')
# papers_df = papers0_df.append(papers1_df)
# papers_df = papers_df.drop_duplicates()
# papers_df.to_csv('./data/papers.csv', index=False)

In [16]:
# papers_df = pd.read_csv('./data/papers.csv')

In [17]:
# papers_df_backup = papers_df.copy()
papers_df = papers_df_backup.copy()

# Merge

In [18]:
def convert_to_list(x):
    """Convert string column to list"""
    try:
        result = literal_eval(x)
    except ValueError:
        result = np.nan
    return result

# Convert strings to list of coauthors
papers_df['orcids'] = papers_df['orcids'].apply(lambda x: convert_to_list(x))

def belongs_to_list(row, df):
    try:
        result = bool(set(row['orcids']) & set(df))
    except TypeError:
        result = False
    return result

# Identify authors in institution
mask = papers_df.apply(lambda x: belongs_to_list(x, res_IGTP), axis=1)

selected_df = papers_df[mask]

In [ ]:
# Save papers of IGTP researchers
# selected_df.to_csv('./data/nodelist_IGTP.csv', index=None)

# Todo
1. Create boolean vector P for each paper of len(authors) that identifies coauthors.
2. Create boolean vector C of coauthor focus. inner(C,P) = 1 if there is collaboration.

### Create bollean vector P for each paper

In [19]:
# Load papers
selected_df = pd.read_csv('./data/nodelist_IGTP.csv', converters = {'orcids': eval})

# selected_df['orcids'] = selected_df['orcids'].apply(lambda x: convert_to_list(x))

# Test with n papers for debugging (max 2800)
n = 100
selected_df = selected_df[:n]

# Get papers column
papers = selected_df['orcids'].copy()
papers = papers.reset_index(drop=True)

# Get unique list of authors from papers
authors_index = list(set(papers.sum()))
authors_index.sort()

# Create boolean matrix with papers
paper_bool_df = pd.DataFrame(columns=authors_index, index=range(len(papers)))

for i, paper in enumerate(papers):
    paper_bool_df.loc[i,:] = 0
    for orcid in paper:
        paper_bool_df.loc[i,orcid] = 1

### Create boolean matrix of coauthor combinations

In [21]:
# Create papers matrix in numpy
papers_mat = paper_bool_df.to_numpy()

# Creation collaboration vector to store results
n_authors = len(authors_index)
collabs = np.zeros(shape=(n_authors*n_authors))

# Main loop to calculate collaborations
# Initialize matrix
C = np.identity(n_authors)
    
for i in range(1, n_authors-1): #last author loop is unnecessary
    print(f"Progress: {i/(n_authors-2)*100:.0f}%.", end="\r")
    
    # Main inner product
    result = np.dot(papers_mat, C)

    # Calculate number of collaborations
    result = result-1
    result = result.clip(0)
    collabs_author = result.sum(axis=0)

    # Store in collabs vector
    collabs[i*n_authors:(i+1)*n_authors] = collabs_author

    # Update combination matrix for next loop
    C[i] = 0
    C[i+1] = 1
    # Set columns
    for j in range(i+1):
        C[:,j] = 0

In [68]:
collabs.sum()

591.0

In [ ]:
# Save collaborations
outfile = pd.DataFrame(links)
outfile.to_csv('./data/edgelist_IGTP.csv')

# EXTRA CODE